In [0]:
  
__author__ = "Emre Girgin"
__email__ = "emregirgin171@gmail.com"
__date__ = "13/Oct/19"

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms  
import numpy as np

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:
transform = transforms.Compose(
    [transforms.ToTensor()] )

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [0]:
class AlexNet(nn.Module):
  def __init__(self):
    super().__init__()
    # padding=same -> p=(f-1)/2
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=96, kernel_size=(11,11), padding=5 )
    self.conv2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=(5,5), padding=2 )
    self.conv3 = nn.Conv2d(in_channels=256, out_channels=384, kernel_size=(3,3), padding=1 )
    self.conv4 = nn.Conv2d(in_channels=384, out_channels=384, kernel_size=(3,3), padding=1 )
    self.conv5 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=(3,3), padding=1 )
    self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(3,3), padding=1 )
    self.fc1 = nn.Linear(in_features=256 * 2 * 2, out_features=4096 )
    self.fc2 = nn.Linear(in_features=4096, out_features=4096 )
    self.fc3 = nn.Linear(in_features=4096, out_features=10 )
  
  def forward(self, x):
    x = self.pool(F.normalize(F.relu(self.conv1(x) ) ) ) # LAYER 1
    x = self.pool(F.normalize(F.relu(self.conv2(x) ) ) ) # LAYER 2
    x = F.relu(self.conv3(x) ) # LAYER 3
    x = F.relu(self.conv4(x) ) # LAYER 4
    x = self.pool(F.relu(self.conv5(x) ) ) # LAYER 5
    x = x.view(-1, 256 * 2 * 2) # FLATTEN
    x = F.relu(self.fc1(x) ) # LAYER 6
    x = F.relu(self.fc2(x) ) # LAYER 7
    x = self.fc3(x) # OUTPUT LAYER
    return x

net = AlexNet() 
print(net)
#net.to(device)

AlexNet(
  (conv1): Conv2d(3, 96, kernel_size=(11, 11), stride=(1, 1), padding=(5, 5))
  (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(3, 3), padding=1, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=4096, bias=True)
  (fc3): Linear(in_features=4096, out_features=10, bias=True)
)


In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [0]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # clear the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        
        # calculate loss
        loss = criterion(outputs, labels)
        
        # backprop 
        loss.backward()

        # update weights
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print("Epochs: {}, Iteration: {}, Loss: {}".format(epoch+1, i, running_loss/100))
            running_loss = 0.0

print('Finished Training')

Epochs: 1, Iteration: 99, Loss: 2.3019249081611632
Epochs: 1, Iteration: 199, Loss: 2.302964470386505


Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: ignored